In [787]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline

In [788]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [789]:
import pandas as pd

#Read input data
train_data = pd.read_csv('/content/drive/MyDrive/ML/titanic-ml-project/data/processed/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ML/titanic-ml-project/data/processed/test.csv')


In [790]:
# Define features and target variable
X_train = train_data.drop('Survived', axis=1)
Y_train = train_data['Survived']

X_test = test_data.drop('Survived', axis=1)
Y_test = test_data['Survived']

## Decision Tree Classifier

In [799]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Initialize the Decision Tree Classifier with optimized parameters
dt = DecisionTreeClassifier(max_depth=18, max_features='sqrt', min_samples_leaf=0.01, min_samples_split=0.061, random_state=42)

# Train the model
dt.fit(X_train, Y_train)

# Make predictions on the test set
Y_dt_opt_pred = dt.predict(X_test)

target_names = ['Didnt Survived', 'Survived']
print(classification_report(Y_test, Y_dt_opt_pred, target_names=target_names))

# Evaluate the model
accuracy = dt.score(X_test, Y_test)
print("Test accuracy:", accuracy)

                precision    recall  f1-score   support

Didnt Survived       0.92      0.98      0.95       266
      Survived       0.97      0.86      0.91       152

      accuracy                           0.94       418
     macro avg       0.95      0.92      0.93       418
  weighted avg       0.94      0.94      0.94       418

Test accuracy: 0.937799043062201


## HistGradientBoostingClassifier

In [800]:
from sklearn.ensemble import HistGradientBoostingClassifier
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Initialize and fit the model with the best parameters
model = HistGradientBoostingClassifier(
    min_samples_leaf=10,
    max_iter=500,
    max_depth=5,
    max_bins=32,
    learning_rate=0.1,
    early_stopping=True
)
model.fit(X_train, Y_train)

# Make predictions
Y_pred = model.predict(X_test)

# Create a DataFrame to compare actual and predicted values
diff_df = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
print(diff_df)

# Compute the confusion matrix
conf_matrix = confusion_matrix(Y_test, Y_pred)
print(conf_matrix)

# Compute the classification report
target_names = ['Didnt Survived', 'Survived']
print(classification_report(Y_test, Y_pred, target_names=target_names))


     Actual  Predicted
0         0          0
1         1          0
2         0          0
3         0          0
4         1          0
..      ...        ...
413       0          0
414       1          1
415       0          0
416       0          0
417       0          0

[418 rows x 2 columns]
[[261   5]
 [ 26 126]]
                precision    recall  f1-score   support

Didnt Survived       0.91      0.98      0.94       266
      Survived       0.96      0.83      0.89       152

      accuracy                           0.93       418
     macro avg       0.94      0.91      0.92       418
  weighted avg       0.93      0.93      0.92       418



## XGBoost

In [801]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the XGBoost model with the specified hyperparameters
model = xgb.XGBClassifier(
    subsample=1,
    reg_lambda=0,
    reg_alpha=0,
    n_estimators=500,
    max_depth=3,
    learning_rate=0.1,
    gamma=1,
    colsample_bytree=0.8,
    eval_metric='mlogloss'  # Change this based on your task (e.g., 'logloss' for binary classification)
)

# Fit the model on the training data
model.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Generate and print the classification report
report = classification_report(Y_test, y_pred)
print("Classification Report:")
print(report)


Accuracy: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92       266
           1       0.94      0.76      0.84       152

    accuracy                           0.90       418
   macro avg       0.91      0.87      0.88       418
weighted avg       0.90      0.90      0.89       418



## Random Forests

## Tunning

#### DecisionTreeClassifier Tunning

In [794]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from scipy.stats import randint, uniform

# Assuming X_train, X_test, Y_train, Y_test are already loaded and split

# Define the parameter distribution for hyperparameter tuning with RandomizedSearchCV
param_dist = {
    'max_depth': randint(1, 21),  # Randomly sample depths from 1 to 20
    'min_samples_split': uniform(0.01, 0.5),  # Uniform distribution between 0.01 and 0.5
    'min_samples_leaf': uniform(0.01, 0.5),  # Uniform distribution between 0.01 and 0.5
    'max_features': ['auto', 'sqrt', 'log2']  # Example values for max_features
}

# Initialize the Decision Tree Classifier
dt = DecisionTreeClassifier(criterion='entropy', random_state=42)

# Use RandomizedSearchCV for hyperparameter tuning with cross-validation
random_search = RandomizedSearchCV(estimator=dt, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, Y_train)

# Best parameters
best_params = random_search.best_params_
print("Best parameters (RandomizedSearchCV):")
for param, value in best_params.items():
    if param in ['min_samples_split', 'min_samples_leaf']:
        print(f"{param}: {round(value, 3)}")
    else:
        print(f"{param}: {value}")

# Define a grid of hyperparameters for GridSearchCV around the best parameters
param_grid = {
    'max_depth': [best_params['max_depth']-2, best_params['max_depth']-1, best_params['max_depth'], best_params['max_depth']+1, best_params['max_depth']+2],
    'min_samples_split': [max(0.01, best_params['min_samples_split']-0.05), round(best_params['min_samples_split'], 3), min(0.5, best_params['min_samples_split']+0.05)],
    'min_samples_leaf': [max(0.01, best_params['min_samples_leaf']-0.05), round(best_params['min_samples_leaf'], 3), min(0.5, best_params['min_samples_leaf']+0.05)],
    'max_features': [best_params['max_features']]
}

# Use GridSearchCV for a more exhaustive search in the promising region
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, Y_train)

# Best parameters
best_params_grid = grid_search.best_params_
print("\nBest parameters (GridSearchCV):")
for param, value in best_params_grid.items():
    if param in ['min_samples_split', 'min_samples_leaf']:
        print(f"{param}: {round(value, 3)}")
    else:
        print(f"{param}: {value}")

# Make predictions on the test set using the best estimator
Y_dt_opt_pred = grid_search.best_estimator_.predict(X_test)

# Define target names for the classification report
target_names = ['Did not Survive', 'Survived']

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_dt_opt_pred, target_names=target_names))

# Evaluate the model's accuracy
accuracy = grid_search.best_estimator_.score(X_test, Y_test)
print("\nTest accuracy:", accuracy)


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
170 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
170 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/util

Best parameters (RandomizedSearchCV):
max_depth: 20
max_features: sqrt
min_samples_leaf: 0.015
min_samples_split: 0.061

Best parameters (GridSearchCV):
max_depth: 18
max_features: sqrt
min_samples_leaf: 0.01
min_samples_split: 0.061

Classification Report:
                 precision    recall  f1-score   support

Did not Survive       0.84      1.00      0.92       266
       Survived       1.00      0.68      0.81       152

       accuracy                           0.88       418
      macro avg       0.92      0.84      0.86       418
   weighted avg       0.90      0.88      0.88       418


Test accuracy: 0.8827751196172249


#### XGBoost Tunning

In [795]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import uniform, randint
import numpy as np

# Define the hyperparameter search space using continuous distributions
param_distributions = {
    'max_depth': randint(3, 10),  # Random integers between 3 and 9
    'learning_rate': uniform(0.01, 0.99),  # Uniform distribution from 0.01 to 1
    'n_estimators': randint(50, 501),  # Random integers between 50 and 500
    'gamma': uniform(0, 1),  # Uniform distribution from 0 to 1
    'subsample': uniform(0.5, 0.5),  # Uniform distribution from 0.5 to 1
    'colsample_bytree': uniform(0.5, 0.5),  # Uniform distribution from 0.5 to 1
    'reg_alpha': uniform(0, 1),  # Uniform distribution from 0 to 1
    'reg_lambda': uniform(0, 1)  # Uniform distribution from 0 to 1
}

# Define the model and the cross-validation strategy
model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform randomized search with cross-validation
random_search = RandomizedSearchCV(
    model,
    param_distributions,
    cv=cv,
    n_iter=30,  # Increased iterations for better exploration
    random_state=42,
    scoring='f1_macro',
    n_jobs=-1,  # Use all available cores
    verbose=1    # Monitor progress
)

random_search.fit(X_train, Y_train)

# Round the best parameters from RandomizedSearchCV
best_params_random = {k: round(v, 2) if isinstance(v, float) else v for k, v in random_search.best_params_.items()}
print("Best parameters from RandomizedSearchCV (rounded):", best_params_random)
print("Best score from RandomizedSearchCV:", random_search.best_score_)

# Define a narrower grid around the best parameters found
param_grid = {
    'max_depth': [best_params_random['max_depth'] - 1, best_params_random['max_depth'], best_params_random['max_depth'] + 1],
    'learning_rate': [round(best_params_random['learning_rate'] - 0.05, 3),
                      round(best_params_random['learning_rate'], 3),
                      round(best_params_random['learning_rate'] + 0.05, 3)],
    'n_estimators': [best_params_random['n_estimators'] - 50, best_params_random['n_estimators'], best_params_random['n_estimators'] + 50],
    'gamma': [round(best_params_random['gamma'] - 0.1, 3),
              round(best_params_random['gamma'], 3),
              round(best_params_random['gamma'] + 0.1, 3)],
    'subsample': [round(best_params_random['subsample'], 3)],
    'colsample_bytree': [round(best_params_random['colsample_bytree'], 3)],
    'reg_alpha': [round(best_params_random['reg_alpha'], 3)],
    'reg_lambda': [round(best_params_random['reg_lambda'], 3)]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,  # Use all available cores
    verbose=1    # Monitor progress
)

grid_search.fit(X_train, Y_train)

# Round the best parameters from GridSearchCV
best_params_grid = {k: round(v, 2) if isinstance(v, float) else v for k, v in grid_search.best_params_.items()}
print("Best parameters from GridSearchCV (rounded):", best_params_grid)
print("Best score from GridSearchCV:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Test set accuracy:", accuracy_score(Y_test, y_pred))
print("Test set classification report:")
print(classification_report(Y_test, y_pred))
print("Test set confusion matrix:")
print(confusion_matrix(Y_test, y_pred))


Fitting 5 folds for each of 30 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import uniform, randint
import numpy as np

def xgboost_hyperparameter_tuning(X_train, Y_train, X_test, Y_test):
    # Define the hyperparameter search space using continuous distributions
    param_distributions = {
        'max_depth': randint(3, 10),
        'learning_rate': uniform(0.01, 0.99),
        'n_estimators': randint(50, 501),
        'gamma': uniform(0, 1),
        'subsample': uniform(0.1, 0.9),
        'colsample_bytree': uniform(0.1, 0.9),
        'reg_alpha': uniform(0, 1),
        'reg_lambda': uniform(0, 1)
    }

    # Define the model and the cross-validation strategy
    model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Perform randomized search with cross-validation
    random_search = RandomizedSearchCV(
        model,
        param_distributions,
        cv=cv,
        n_iter=50,
        random_state=42,
        scoring='f1_macro',
        n_jobs=-1,
        verbose=1
    )

    random_search.fit(X_train, Y_train)

    # Round the best parameters from RandomizedSearchCV
    best_params_random = {k: round(v, 2) if isinstance(v, float) else v for k, v in random_search.best_params_.items()}
    print("Best parameters from RandomizedSearchCV (rounded):", best_params_random)
    print("Best score from RandomizedSearchCV:", random_search.best_score_)

    # Define a narrower grid around the best parameters found
    param_grid = {
        'max_depth': [best_params_random['max_depth'] - 1, best_params_random['max_depth'], best_params_random['max_depth'] + 1],
        'learning_rate': [round(best_params_random['learning_rate'] - 0.05, 3),
                          round(best_params_random['learning_rate'], 3),
                          round(best_params_random['learning_rate'] + 0.05, 3)],
        'n_estimators': [best_params_random['n_estimators'] - 50, best_params_random['n_estimators'], best_params_random['n_estimators'] + 50],
        'gamma': [round(best_params_random['gamma'] - 0.1, 3),
                  round(best_params_random['gamma'], 3),
                  round(best_params_random['gamma'] + 0.1, 3)],
        'subsample': [round(best_params_random['subsample'], 3)],
        'colsample_bytree': [round(best_params_random['colsample_bytree'], 3)],
        'reg_alpha': [round(best_params_random['reg_alpha'], 3)],
        'reg_lambda': [round(best_params_random['reg_lambda'], 3)]
    }

    # Perform grid search with cross-validation
    grid_search = GridSearchCV(
        model,
        param_grid,
        cv=cv,
        scoring='f1_macro',
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(X_train, Y_train)

    # Round the best parameters from GridSearchCV
    best_params_grid = {k: round(v, 2) if isinstance(v, float) else v for k, v in grid_search.best_params_.items()}
    print("Best parameters from GridSearchCV (rounded):", best_params_grid)
    print("Best score from GridSearchCV:", grid_search.best_score_)

    # Evaluate the best model on the test set
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    print("Test set accuracy:", accuracy_score(Y_test, y_pred))
    print("Test set classification report:")
    print(classification_report(Y_test, y_pred))
    print("Test set confusion matrix:")
    print(confusion_matrix(Y_test, y_pred))

# Assuming X_train, X_test, Y_train, Y_test are already loaded and split
xgboost_hyperparameter_tuning(X_train, Y_train, X_test, Y_test)


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the XGBoost model with the specified hyperparameters
model = xgb.XGBClassifier(
    subsample=0.53,
    reg_lambda=0.5,
    reg_alpha=0.61,
    n_estimators=236,
    max_depth=2,
    learning_rate=0.16,
    gamma=0.71,
    colsample_bytree=0.85,
    use_label_encoder=False,
    eval_metric='mlogloss'  # Change this based on your task (e.g., 'logloss' for binary classification)
)

# Fit the model on the training data
model.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Generate and print the classification report
report = classification_report(Y_test, y_pred)
print("Classification Report:")
print(report)


#### HistGradientBoostingClassifier Tunning

In [ ]:
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from scipy.stats import uniform, randint

# Create a synthetic dataset
X, Y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter search space using continuous distributions
param_dist = {
    'learning_rate': uniform(0.01, 1),  # Continuous values between 0.01 and 1
    'max_iter': randint(100, 1001),      # Integer values between 100 and 1000
    'max_depth': randint(3, 8),          # Integer values between 3 and 7
    'min_samples_leaf': randint(1, 11),  # Integer values between 1 and 10
    'max_bins': randint(16, 65),         # Integer values between 16 and 64
    'early_stopping': [True, False]
}

# Define the model and the cross-validation strategy
model = HistGradientBoostingClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform randomized search with cross-validation
random_search = RandomizedSearchCV(model, param_dist, cv=cv, n_iter=30, random_state=42)
random_search.fit(X_train, Y_train)

# Print the best parameters and the best score
print("Best parameters (before rounding):", random_search.best_params_)
print("Best score:", random_search.best_score_)

# Round the best parameters to 3 decimal places
best_params_rounded = {k: round(v, 3) if isinstance(v, float) else v for k, v in random_search.best_params_.items()}
print("Best parameters (after rounding):", best_params_rounded)

# Define a smaller parameter grid for RandomizedSearchCV based on the best parameters found
grid_param = {
    'learning_rate': [best_params_rounded['learning_rate'] - 0.01, best_params_rounded['learning_rate'], best_params_rounded['learning_rate'] + 0.01],
    'max_iter': [best_params_rounded['max_iter'] - 100, best_params_rounded['max_iter'], best_params_rounded['max_iter'] + 100],
    'max_depth': [best_params_rounded['max_depth'] - 1, best_params_rounded['max_depth'], best_params_rounded['max_depth'] + 1],
    'min_samples_leaf': [best_params_rounded['min_samples_leaf'] - 1, best_params_rounded['min_samples_leaf'], best_params_rounded['min_samples_leaf'] + 1],
    'max_bins': [best_params_rounded['max_bins'] - 8, best_params_rounded['max_bins'], best_params_rounded['max_bins'] + 8],
    'early_stopping': [True, False]
}

# Perform randomized search again instead of grid search for efficiency
random_search_final = RandomizedSearchCV(model, grid_param, cv=cv, n_iter=30, n_jobs=-1, random_state=42)
random_search_final.fit(X_train, Y_train)

# Print the best parameters and the best score from the final search
print("Best parameters from final RandomizedSearchCV:", random_search_final.best_params_)
print("Best score from final RandomizedSearchCV:", random_search_final.best_score_)

# Evaluate the best model on the test set
best_model = random_search_final.best_estimator_
test_score = best_model.score(X_test, Y_test)
print("Test set score:", test_score)


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Initialize and fit the model with the best parameters
model = HistGradientBoostingClassifier(
    min_samples_leaf=7,
    max_iter=558,
    max_depth=4,
    max_bins=38,
    learning_rate=0.456,
    early_stopping=True
)
model.fit(X_train, Y_train)

# Make predictions
Y_pred = model.predict(X_test)

# Create a DataFrame to compare actual and predicted values
diff_df = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
print(diff_df)

# Compute the confusion matrix
conf_matrix = confusion_matrix(Y_test, Y_pred)
print(conf_matrix)

# Compute the classification report
target_names = ['Didnt Survived', 'Survived']
print(classification_report(Y_test, Y_pred, target_names=target_names))